In [26]:
import os
import sys
ROOT = os.path.dirname(os.getcwd())
path_data = 'C:\\Users\\Gosia\\Desktop\\'
sys.path.insert(0, ROOT)

In [27]:
# Using MatchMS to create spectra
from matchms.importing import load_from_json 
specs =  load_from_json(r'C:\Users\Gosia\Desktop\gnps_from_simon\0000.txt')

In [28]:
from matchms.filtering import normalize_intensities
# Spec2Vec trained model requires normalizing
specs = [normalize_intensities(s) for s in specs]


In [29]:
import gensim
path_models = os.path.join(path_data, "trained_models")
model_file = os.path.join(path_models, "spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model")

# Load pretrained model
model = gensim.models.Word2Vec.load(model_file)

In [30]:
from spec2vec import Spec2Vec
from spec2vec import SpectrumDocument

documents = [SpectrumDocument(s, n_decimals=2) for s in specs]
print(documents)


[<spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7820>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A77C0>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7790>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7700>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A76D0>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x00000285AED512B0>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7580>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A75B0>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7550>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A74F0>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7490>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A7430>, <spec2vec.SpectrumDocument.SpectrumDocument object at 0x000002859C1A79D0>, <spec2vec.SpectrumDocume

In [76]:
import numpy as np
from tqdm.notebook import tqdm  # optional, just to get a progress bar
from spec2vec.vector_operations import calc_vector


intensity_weighting_power = 0.5
allowed_missing_percentage = 100 # specify the maximum (weighted) fraction of the spectrum that is allowed to be missing

vector_size = model.vector_size
print(f"Embedding vector size: {vector_size}")

embeddings = np.zeros((len(documents), vector_size), dtype="float")
for i, doc in enumerate(tqdm(documents)):
    embeddings[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power=0.5,
                                                        allowed_missing_percentage=allowed_missing_percentage)

Embedding vector size: 300


Found 2 word(s) missing in the model. Weighted missing percentage not covered by the given model is 0.45%.
Found 10 word(s) missing in the model. Weighted missing percentage not covered by the given model is 5.28%.
Found 16 word(s) missing in the model. Weighted missing percentage not covered by the given model is 54.38%.
Found 1 word(s) missing in the model. Weighted missing percentage not covered by the given model is 0.20%.
Found 1 word(s) missing in the model. Weighted missing percentage not covered by the given model is 10.93%.
Found 7 word(s) missing in the model. Weighted missing percentage not covered by the given model is 5.97%.
Found 1 word(s) missing in the model. Weighted missing percentage not covered by the given model is 0.88%.
Found 1 word(s) missing in the model. Weighted missing percentage not covered by the given model is 0.63%.
Found 1 word(s) missing in the model. Weighted missing percentage not covered by the given model is 0.31%.
Found 77 word(s) missing in the m

In [77]:
len(specs), len(embeddings)
xs = [ ( s.metadata['parent_mass'], i ) for i, s in enumerate( specs ) ]
prev_mass = 0
for mass, idx in sorted(xs):
    print(mass-prev_mass, idx)
    prev_mass = mass

313.1087235480092 4
9.141999999999996 68
21.982000000000028 69
2.9149999999999636 53
3.0270000000000437 10
21.98199999999997 12
23.110000000000014 94
1.9700000000000273 64
5.886999999999944 72
6.1510000000000105 76
0.0 78
7.9739999999999895 96
0.8990000000000009 80
1.0710000000000264 65
3.774000000000001 40
0.8999999999999773 52
1.302000000000021 73
0.7980000000000018 50
5.26400000000001 77
5.887999999999977 14
1.9360000000000355 62
1.0489999999999782 82
3.975999999999999 98
10.987000000000023 2
1.9979999999999905 63
2.901999999999987 28
2.0 26
21.0 32
5.0 22
2.0 20
6.899999999999977 36
0.10000000000002274 46
0.10000000000002274 38
12.899999999999977 24
2.3080000000000496 27
2.01400000000001 43
16.677999999999997 30
3.2899999999999636 29
2.01400000000001 45
7.696000000000026 3
21.427999999999997 41
0.5720000000000027 17
2.0 13
9.33299999999997 37
5.951999999999998 47
4.715000000000032 18
2.0 15
9.313999999999965 39
1.0439999999999827 74
4.9090000000001055 49
1.6329999999999245 42
0.100

In [74]:
specs[76].metadata['compound_name'], specs[78].metadata['compound_name']

('Marineosin A', 'Marineosin B')

In [78]:
a = [np.round(x, 4) for x in embeddings[76,:]]
b = [np.round(x, 4) for x in embeddings[78,:]]
#c = [np.round(x, 4) for x in embeddings[2,:]]
for v in zip(a,b):#,c):
    print('%.3f'%abs(v[0]-v[1]),  v)

2.922 (8.0201, 5.0983)
11.133 (-27.3661, -16.2333)
7.931 (-18.2498, -10.3191)
3.671 (-7.5686, -11.24)
1.225 (-15.7997, -14.5748)
10.940 (21.0263, 10.0863)
0.929 (-12.7952, -11.8658)
3.147 (-1.0138, -4.1613)
5.473 (-1.5228, 3.9506)
5.081 (-5.4385, -10.5194)
1.646 (-4.8993, -3.2538)
0.709 (0.2814, -0.4275)
0.836 (0.7196, 1.5559)
2.642 (2.7374, 5.3792)
3.977 (2.7646, -1.2119)
1.335 (22.4164, 21.0816)
3.021 (15.3035, 12.2828)
1.176 (-3.9623, -5.1387)
3.182 (7.8021, 10.9843)
13.009 (-23.5178, -10.5086)
12.818 (-19.5313, -6.7132)
0.397 (4.8873, 4.4901)
1.376 (-1.5557, -0.1793)
0.492 (19.6788, 19.187)
1.697 (3.3335, 1.6368)
6.728 (-10.0286, -3.3007)
4.546 (-5.9797, -10.5261)
4.859 (4.8069, -0.0525)
0.403 (-4.3474, -3.9447)
0.943 (-7.0553, -6.1126)
5.503 (8.1744, 2.6716)
8.892 (12.8526, 3.9608)
1.232 (16.4073, 15.1755)
6.856 (25.5142, 18.6578)
0.565 (3.7125, 3.1472)
3.257 (-16.7965, -13.5394)
7.556 (-16.7943, -9.2385)
2.589 (-2.2371, 0.3515)
3.329 (3.6112, 6.9404)
1.062 (-2.1671, -1.1047)
1.86

In [71]:
s1, s2 = specs[173], specs[237]
doc1, doc2 = documents[173], documents[237]

IndexError: list index out of range